In [6]:
#Generate the uniform random variables
import math
def Gen_Uniform (n,seed=1):
    result=[]
    for i in range(n):
        seed=(seed*39373)%(2**31-1)
        result.append(seed/(2**31-1))
    return result
#Gen_Uniform(100)

In [7]:
#Generate standard normal random variable with Box-Muller method using Marsaglia-Bray Algo
import matplotlib.pyplot as plt
def Gen_Normal (n,seed=1):
    result=[]
    u=Gen_Uniform(10*n,seed)
    k=0
    for i in range(n):
        x=10
        while x>1:
            u1=u[k]
            u1=2*u1-1
            k=k+1
            u2=u[k]
            u2=2*u2-1
            k=k+1
            x=u1*u1+u2*u2
        y=math.sqrt(-2*math.log(x)/x)
        z1=u1*y
        z2=u2*y
        result.append(z1)
        result.append(z2)
    return result
#Gen_Normal(100)
#plt.hist(nm)

In [8]:
import math
def Heston(m,n,lam=4,Vtilde=0.1225,eta=0.25,rho=-0.15,r=0.05,V0=0.09,S0=50,T=0.5):
    nm=Gen_Normal(m*n)
    dt=T/m
    k=0
    Vhat=0
    Phat=0
    for i in range(n):
        V=V0
        S=S0
        for j in range(m):
            z1=nm[k]
            k+=1
            z2=nm[k]
            k+=1
            S=S*math.exp((r-max(V,0)/2)*dt+math.sqrt(max(V,0)*dt)*z1)
            V=max(V,0)-lam*(max(V,0)-Vtilde)*dt+eta*math.sqrt(max(V,0)*dt)*(rho*z1+math.sqrt(1-rho*rho)*z2)
        Vhat+=V/n
        Phat+=math.exp(-r*T)*(max(S0-S,0))/n
    return [Vhat,Phat]



In [9]:
import scipy as sp
n = sp.stats.norm.pdf
N = sp.stats.norm.cdf
def bs_price(cp_flag,S,K,T,r,v,q=0.0):
    d1 = (math.log(S/K)+(r+v*v/2.)*T)/(v*math.sqrt(T))
    d2 = d1-v*math.sqrt(T)
    if cp_flag == 'c':
        price = S*math.exp(-q*T)*N(d1)-K*math.exp(-r*T)*N(d2)
    else:
        price = K*math.exp(-r*T)*N(-d2)-S*math.exp(-q*T)*N(-d1)
    return price
def bs_vega(cp_flag,S,K,T,r,v,q=0.0):
    d1 = (math.log(S/K)+(r+v*v/2.)*T)/(v*math.sqrt(T))
    return S * math.sqrt(T)*n(d1)

def find_vol(target_value, call_put, S, K, T, r):
    MAX_ITERATIONS = 100
    PRECISION = 1.0e-7
    sigma = 0.5
    for i in range(0, MAX_ITERATIONS):
        price = bs_price(call_put, S, K, T, r, sigma)
        vega = bs_vega(call_put, S, K, T, r, sigma)
        price = price
        diff = target_value - price  
        #print(i, sigma, diff)
        if (abs(diff) < PRECISION):
            return sigma
        sigma = sigma + diff/vega # f(x) / f'(x)
    return 0

for k in range(6):
    res=Heston(175,500*(2**k))
    print("m = 175, n =",500*(2**k))
    print("V:",res[0])
    print("P:",res[1])
    print("implied volatility:",find_vol(res[1],'p',50,50,0.5,0.05))

m = 175, n = 500
V: 0.11796429864713613
P: 3.9588521348855616
implied volatility: 0.3273270847687851
m = 175, n = 1000
V: 0.11769844441374623
P: 3.735216769947576
implied volatility: 0.3110702494853444
m = 175, n = 2000
V: 0.11888479617198656
P: 3.9614099515942387
implied volatility: 0.32751302562311424
m = 175, n = 4000
V: 0.11873516536415112
P: 4.02563823754003
implied volatility: 0.33218217845300707
m = 175, n = 8000
V: 0.11851295646366804
P: 4.014398561331073
implied volatility: 0.33136508618995303
m = 175, n = 16000
V: 0.11815218417463297
P: 3.981491257802374
implied volatility: 0.3289728457418796
